# Submission - Inquisitive_Turtles

In [1]:
import kagglehub
path = kagglehub.dataset_download("schran/insurance-premium-prediction")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/insurance-premium-prediction


In [2]:
import pandas as pd
df = pd.read_csv('/kaggle/input/insurance-premium-prediction/Insurance Premium Prediction Dataset.csv')
print(df.columns,df.dtypes)

Index(['Age', 'Gender', 'Annual Income', 'Marital Status',
       'Number of Dependents', 'Education Level', 'Occupation', 'Health Score',
       'Location', 'Policy Type', 'Previous Claims', 'Vehicle Age',
       'Credit Score', 'Insurance Duration', 'Premium Amount',
       'Policy Start Date', 'Customer Feedback', 'Smoking Status',
       'Exercise Frequency', 'Property Type'],
      dtype='object') Age                     float64
Gender                   object
Annual Income           float64
Marital Status           object
Number of Dependents    float64
Education Level          object
Occupation               object
Health Score            float64
Location                 object
Policy Type              object
Previous Claims         float64
Vehicle Age               int64
Credit Score            float64
Insurance Duration        int64
Premium Amount          float64
Policy Start Date        object
Customer Feedback        object
Smoking Status           object
Exercise Frequency

In [3]:
df.describe()

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,274175.000000,264905.000000,250974.000000,268263.000000,197572.000000,278860.000000,250974.000000,278860.000000,277019.000000
mean,41.020771,42089.085329,1.998048,28.584290,0.998117,9.520283,574.362049,5.007764,966.118667
std,13.549683,35444.517255,1.412312,15.966208,1.000795,5.767915,158.792037,2.581349,909.404567
min,18.000000,0.000000,0.000000,0.035436,0.000000,0.000000,300.000000,1.000000,0.000000
25%,29.000000,13588.000000,1.000000,16.149890,0.000000,5.000000,437.000000,3.000000,286.000000
50%,41.000000,32191.000000,2.000000,26.451244,1.000000,10.000000,575.000000,5.000000,688.000000
75%,53.000000,62164.000000,3.000000,38.966369,2.000000,15.000000,712.000000,7.000000,1367.000000
max,64.000000,149997.000000,4.000000,93.876090,9.000000,19.000000,849.000000,9.000000,4999.000000


In [4]:
missing_values = df.isnull().sum()
cols_with_missing = missing_values[missing_values > 0]
print(cols_with_missing)

Age                      4685
Annual Income           13955
Marital Status           5019
Number of Dependents    27886
Occupation              81288
Health Score            10597
Previous Claims         81288
Credit Score            27886
Premium Amount           1841
Customer Feedback       18349
dtype: int64


Getting rid of null values in target column

In [5]:
df = df.dropna(subset = 'Premium Amount')

In [6]:
missing_values = df.isnull().sum()
cols_with_missing = missing_values[missing_values > 0]
print(cols_with_missing)

Age                      4658
Annual Income           13851
Marital Status           4991
Number of Dependents    27689
Occupation              80748
Health Score            10525
Previous Claims         80749
Credit Score            27700
Customer Feedback       18258
dtype: int64


In [7]:
Q1 = df['Previous Claims'].quantile(0.25)
Q3 = df['Previous Claims'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_cleaned = df[(df['Previous Claims'] >= lower_bound) & (df['Previous Claims'] <= upper_bound)]
df_outliers = df[(df['Previous Claims'] < lower_bound) | (df['Previous Claims'] > upper_bound)]

print('number of outliers: ',df_outliers['Previous Claims'].count())
print("Original DataFrame with outliers:",df['Previous Claims'].count())
print("DataFrame after removing outliers:",df_cleaned['Previous Claims'].count())

number of outliers:  117
Original DataFrame with outliers: 196270
DataFrame after removing outliers: 196153


In [8]:
mean_imputation_cols = ['Age','Number of Dependents','Credit Score','Credit Score']

In [9]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')

df_cleaned[mean_imputation_cols]= imputer.fit_transform(df_cleaned[mean_imputation_cols])
print(df_cleaned)



         Age  Gender  Annual Income Marital Status  Number of Dependents  \
2       32.0  Female        30154.0       Divorced                   3.0   
3       60.0  Female        48371.0       Divorced                   0.0   
5       38.0    Male        31769.0        Married                   1.0   
6       56.0  Female        26209.0         Single                   1.0   
7       36.0    Male         8762.0         Single                   3.0   
...      ...     ...            ...            ...                   ...   
278851  25.0  Female         4630.0       Divorced                   4.0   
278852  47.0  Female        14001.0         Single                   0.0   
278853  28.0  Female        74931.0        Married                   0.0   
278854  63.0    Male        34311.0       Divorced                   0.0   
278855  53.0    Male        22594.0       Divorced                   4.0   

       Education Level     Occupation  Health Score  Location    Policy Type  \
2      

<ipython-input-9-a269f45f3bbf>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[mean_imputation_cols]= imputer.fit_transform(df_cleaned[mean_imputation_cols])


In [10]:
missing_values = df_cleaned.isnull().sum()
check_missing = missing_values[missing_values > 0]
print(check_missing)

Annual Income         9773
Marital Status        3490
Occupation           57166
Health Score          7447
Customer Feedback    12959
dtype: int64


In [11]:
mode_imputation_cols=['Annual Income','Health Score']

In [12]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')

df_cleaned[mode_imputation_cols]= imputer.fit_transform(df_cleaned[mode_imputation_cols])
print(df_cleaned)


         Age  Gender  Annual Income Marital Status  Number of Dependents  \
2       32.0  Female        30154.0       Divorced                   3.0   
3       60.0  Female        48371.0       Divorced                   0.0   
5       38.0    Male        31769.0        Married                   1.0   
6       56.0  Female        26209.0         Single                   1.0   
7       36.0    Male         8762.0         Single                   3.0   
...      ...     ...            ...            ...                   ...   
278851  25.0  Female         4630.0       Divorced                   4.0   
278852  47.0  Female        14001.0         Single                   0.0   
278853  28.0  Female        74931.0        Married                   0.0   
278854  63.0    Male        34311.0       Divorced                   0.0   
278855  53.0    Male        22594.0       Divorced                   4.0   

       Education Level     Occupation  Health Score  Location    Policy Type  \
2      

<ipython-input-12-3e9759c0bb16>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[mode_imputation_cols]= imputer.fit_transform(df_cleaned[mode_imputation_cols])


In [13]:
missing_values = df_cleaned.isnull().sum()
check_missing = missing_values[missing_values > 0]
print(check_missing)

Marital Status        3490
Occupation           57166
Customer Feedback    12959
dtype: int64


In [14]:
mode_imputation_cols=['Marital Status','Occupation']

In [15]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')

df_cleaned[mode_imputation_cols]= imputer.fit_transform(df_cleaned[mode_imputation_cols])
print(df_cleaned)


         Age  Gender  Annual Income Marital Status  Number of Dependents  \
2       32.0  Female        30154.0       Divorced                   3.0   
3       60.0  Female        48371.0       Divorced                   0.0   
5       38.0    Male        31769.0        Married                   1.0   
6       56.0  Female        26209.0         Single                   1.0   
7       36.0    Male         8762.0         Single                   3.0   
...      ...     ...            ...            ...                   ...   
278851  25.0  Female         4630.0       Divorced                   4.0   
278852  47.0  Female        14001.0         Single                   0.0   
278853  28.0  Female        74931.0        Married                   0.0   
278854  63.0    Male        34311.0       Divorced                   0.0   
278855  53.0    Male        22594.0       Divorced                   4.0   

       Education Level     Occupation  Health Score  Location    Policy Type  \
2      

<ipython-input-15-3e9759c0bb16>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[mode_imputation_cols]= imputer.fit_transform(df_cleaned[mode_imputation_cols])


In [16]:
missing_values = df_cleaned.isnull().sum()
check_missing = missing_values[missing_values > 0]
print(check_missing)

Customer Feedback    12959
dtype: int64


In [17]:
df_cleaned['Customer Feedback']= df_cleaned['Customer Feedback'].fillna('')

<ipython-input-17-1308beb94693>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Customer Feedback']= df_cleaned['Customer Feedback'].fillna('')


In [18]:
missing_values = df_cleaned.isnull().sum()
check_missing = missing_values[missing_values > 0]
print(check_missing)

Series([], dtype: int64)


In [19]:
df_cleaned.dtypes

Age                     float64
Gender                   object
Annual Income           float64
Marital Status           object
Number of Dependents    float64
Education Level          object
Occupation               object
Health Score            float64
Location                 object
Policy Type              object
Previous Claims         float64
Vehicle Age               int64
Credit Score            float64
Insurance Duration        int64
Premium Amount          float64
Policy Start Date        object
Customer Feedback        object
Smoking Status           object
Exercise Frequency       object
Property Type            object
dtype: object

# One Hot encoding

In [20]:
from sklearn.preprocessing import OneHotEncoder
encoding_cols=['Customer Feedback','Occupation','Education Level','Marital Status',
               'Property Type','Exercise Frequency','Smoking Status','Policy Type','Gender','Location']

encoder = OneHotEncoder(sparse=False)  # Use sparse=True for sparse matrix output

# Fit and transform the data
encoded_array = encoder.fit_transform(df_cleaned[encoding_cols])

# Convert the encoded array to a DataFrame with column names
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(encoding_cols))

# Drop the original categorical columns and concatenate with the encoded DataFrame
df_cleaned = pd.concat([df_cleaned.drop(columns=encoding_cols), encoded_df], axis=1)

print(df_cleaned)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


         Age  Annual Income  Number of Dependents  Health Score  \
2       32.0        30154.0                   3.0     14.714909   
3       60.0        48371.0                   0.0     25.346926   
5       38.0        31769.0                   1.0     14.736611   
6       56.0        26209.0                   1.0     33.238300   
7       36.0         8762.0                   3.0     13.887880   
...      ...            ...                   ...           ...   
196137   NaN            NaN                   NaN           NaN   
196138   NaN            NaN                   NaN           NaN   
196139   NaN            NaN                   NaN           NaN   
196142   NaN            NaN                   NaN           NaN   
196146   NaN            NaN                   NaN           NaN   

        Previous Claims  Vehicle Age  Credit Score  Insurance Duration  \
2                   2.0         16.0         652.0                 8.0   
3                   1.0         11.0         33